In [ ]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Reshape, Conv2D, MaxPooling2D, UpSampling2D
from keras import optimizers
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
import librosa
from librosa import display
from librosa.output import write_wav
import cv2

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [ ]:
def denormalize(array):
    return array * _max

In [ ]:
X_train = np.load('./data/ravdess-gan-data.npy')
_max = np.amax(X_train)

# normalize
X_train = X_train / _max

In [ ]:
# blur image
X_train_noisy = []
for x in X_train:    
    blurred_image = cv2.GaussianBlur(np.squeeze(x), (5, 5), 5)
    X_train_noisy.append(blurred_image)

In [ ]:
X_train = np.expand_dims(X_train, axis=3)
X_train_noisy = np.array(X_train_noisy)
X_train_noisy = np.expand_dims(X_train_noisy, axis=3)

In [ ]:
autoencoder = Sequential()

autoencoder.add(Conv2D(128, 2, activation='relu', padding='same', input_shape=(128, 64, 1)))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))

autoencoder.add(Conv2D(128, 2, activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))

autoencoder.add(Conv2D(128, 2, activation='relu', padding='same'))

autoencoder.add(Conv2D(64, 2, activation='relu', padding='same'))

autoencoder.add(Conv2D(1, 2, activation='sigmoid', padding='same'))

optim = Adam(lr=.002)
autoencoder.compile(optimizer=optim, loss='binary_crossentropy')

In [ ]:
autoencoder.fit(X_train_noisy, X_train, epochs=500, batch_size=128, validation_split=.1)

In [ ]:
librosa.display.specshow(X_train_noisy[0].reshape(128, 64))

In [ ]:
data = X_train[0]
data = np.expand_dims(data, axis=0)
data = autoencoder.predict(data) 
data = data.reshape(128, 64)
librosa.display.specshow(data)
reconstructed = librosa.feature.inverse.mel_to_audio(denormalize(data), sr=16000, power=.5)
write_wav('reconstructed.wav', sr=16000, y=reconstructed)

In [ ]:
data = X_train[0].reshape(128, 64)
librosa.display.specshow(data)
real = librosa.feature.inverse.mel_to_audio(denormalize(data), sr=16000, power=.5)
write_wav('real.wav', sr=16000, y=real)

In [ ]:
data, rate = librosa.load('./audio/yes_0.wav', duration=2.02, sr=16000,)
spectrogram = librosa.feature.melspectrogram(y=data, sr=rate, n_mels=128, power=.5)
librosa.display.specshow(spectrogram)

In [ ]:
spectrogram = np.expand_dims(spectrogram, axis=0)
spectrogram = np.expand_dims(spectrogram, axis=3)
output = autoencoder.predict(spectrogram)
output = np.squeeze(output)
librosa.display.specshow(output)
output = librosa.feature.inverse.mel_to_audio(denormalize(output), sr=16000, power=.5)
write_wav('test.wav', sr=16000, y=output)